In [1]:
# Enlarging the screen

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 85%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

print ('Enlarging the screen is done!')

# Importing libraries

import numpy
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

pd.set_option('max_colwidth', 100)

pd.options.mode.chained_assignment = None  # default='warn'

print ('Libraries were imported successfully!')

# Loading data from sql

Server = 'LAPTOP-I7NEB9V3\SQLEXPRESS'
Database = 'Geopattern'
Driver = 'ODBC Driver 17 for SQL Server'
Database_Connection = f'mssql://@{Server}/{Database}?driver={Driver}'

engine = create_engine(Database_Connection)
connection = engine.connect()

df_my_data = pd.read_sql_query (
    "select * from my_data", connection)

df_Authors = pd.read_sql_query (
    "select * from my_data_Authors", connection)

df_only_Canada = pd.read_sql_query (
    "select distinct my_data.EID, my_data.Author_ID, my_data.Year, my_data.Country from my_data join \
        (select distinct EID from my_data \
        except \
        (select distinct EID from my_data \
        where Country not like 'Canada' or Country is Null)) a \
        on my_data.EID = a.EID", connection)

df_only_Canada.Year = df_only_Canada.Year.astype(int)

df_LDA = pd.read_csv(r'C:\Users\moham\Dropbox\QSE\Thesis\Geopattern\My data\df_LDA.csv')
df_LDA.set_index('EID', inplace = True)

print ('Loading data from sql is done!')

width_ind = 3
width_dep = 2

df_data_set = pd.DataFrame()

list_years = []

for yrs in range(21 - width_ind - width_dep):
    list_years.append(2000 + yrs)


for yr in tqdm(list_years, desc = 'Preparing the data set'):


    Ind_win_start = yr
    Ind_win_end = Ind_win_start + width_ind - 1

    dep_win_start = Ind_win_end + 1
    dep_win_end = dep_win_start + width_dep - 1

    df_ind = df_only_Canada[(df_only_Canada.Year > (Ind_win_start - 1)) & (df_only_Canada.Year <= Ind_win_end)]
#     df_ind = df_only_Canada[(df_only_Canada.Year <= Ind_win_end)]
    df_dep = df_only_Canada[(df_only_Canada.Year > (dep_win_start - 1)) & (df_only_Canada.Year <= dep_win_end)]

    #     df_ind.rename({'EID' : 'EIDs_ind'}, axis = 1, inplace = True)
    #     df_dep.rename({'EID' : 'EIDs_dep'}, axis = 1, inplace = True)


    # Targets.................................................................................................. 

    list_authors_dep = df_dep.Author_ID.unique().tolist()
    list_authors_ind = df_ind.Author_ID.unique().tolist()
    
    list_authors = list(set(list_authors_dep).intersection(list_authors_ind))
    list_authors.sort()

    df_authors_dep = pd.DataFrame(data = list_authors, columns = ['Author_ID'])
    df_authors_dep.insert(1,'EIDs','')

    # df_authors_dep = df_authors_dep.merge(df_Authors, on = 'Author_ID', how = 'left')

    authors_dep = []
    for i in range (df_authors_dep.shape[0]):
        for j in range (df_authors_dep.shape[0]):
            if j > i:
                authors_dep.append((df_authors_dep['Author_ID'][i] + df_authors_dep['Author_ID'][j], df_authors_dep['Author_ID'][i], df_authors_dep['Author_ID'][j]))

    df_data_set_dep = pd.DataFrame(data = authors_dep, columns=['Author_1_2', 'Author_1', 'Author_2'])

    df_data_set_dep = df_data_set_dep.set_index(['Author_1_2'])

    df_data_set_dep.insert(2,'number_of_collaborations',0)
    df_data_set_dep.insert(3,'collaboration_binary',0)

    df_dep.reset_index(inplace = True)

    df_authors_dep['EIDs'] = ''
    for i in range (df_authors_dep.shape[0]):
        for j in range (df_dep.shape[0]):
            if df_authors_dep['Author_ID'][i] == df_dep['Author_ID'][j]:
                df_authors_dep['EIDs'][i] = str (df_authors_dep['EIDs'][i]) + ';' + str (df_dep['EID'][j])

    res = []

    for i in range (df_authors_dep.shape[0]):
        l = df_authors_dep['EIDs'][i].split(';')
        l.remove('')
        res.append((df_authors_dep['Author_ID'][i],set(l)))


    collab_matrix = np.zeros((df_authors_dep.shape[0],df_authors_dep.shape[0]))

    for i in range (len(res)):
        for j in range (len(res)):
            collab_matrix[i,j] = len(res[i][1].intersection(res[j][1]))

    collab_list = []
    for i in range (collab_matrix.shape[0]):
        for j in range (collab_matrix.shape[0]):
            if j > i:
                if collab_matrix[i,j] != 0:
                    collab_list.append((df_authors_dep['Author_ID'][i] + df_authors_dep['Author_ID'][j], df_authors_dep['Author_ID'][i], df_authors_dep['Author_ID'][j], collab_matrix[i,j]))

    df_collab = pd.DataFrame(data = collab_list, columns=['Author_1_2', 'Author_1', 'Author_2', 'number_of_collaborations'])

    df_collab = df_collab.set_index(['Author_1_2'])


    for i in df_data_set_dep.index:
        try:
            df_data_set_dep.loc[i,'number_of_collaborations'] = df_collab.loc[i,'number_of_collaborations']
        except:
            df_data_set_dep.loc[i,'number_of_collaborations'] = 0


    df_data_set_dep.collaboration_binary = df_data_set_dep.number_of_collaborations.map(lambda x: 1 if x > 0 else 0)


    # Features............................................................................................

    df_authors_ind = pd.DataFrame(data = list_authors, columns = ['Author_ID'])
    df_authors_ind.insert(1,'EIDs','')
    df_authors_ind.insert(2,'partners',0)
    df_authors_ind.insert(3,'topic_1',0)
    df_authors_ind.insert(4,'topic_2',0)
    df_authors_ind.insert(5,'topic_3',0)
    df_authors_ind.insert(6,'topic_4',0)
    df_authors_ind.insert(7,'topic_5',0)
    df_authors_ind.insert(8,'topic_6',0)
    df_authors_ind.insert(9,'topic_7',0)
    df_authors_ind.insert(10,'topic_8',0)
    df_authors_ind.insert(11,'topic_9',0)


    df_authors_ind = df_authors_ind.merge(df_Authors, on = 'Author_ID', how = 'left')

    df_authors_ind.reset_index(inplace = True)
    df_ind.reset_index(inplace = True)

    df_authors_ind['EIDs'] = ''

    for i in range (df_authors_ind.shape[0]):
        for j in range (df_ind.shape[0]):
            if df_authors_ind['Author_ID'][i] == df_ind['Author_ID'][j]:
                df_authors_ind['EIDs'][i] = str (df_authors_ind['EIDs'][i]) + ';' + str (df_ind['EID'][j])
                df_authors_ind.loc[i, 'topic_1'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_1']
                df_authors_ind.loc[i, 'topic_2'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_2']
                df_authors_ind.loc[i, 'topic_3'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_3']
                df_authors_ind.loc[i, 'topic_4'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_4']
                df_authors_ind.loc[i, 'topic_5'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_5']
                df_authors_ind.loc[i, 'topic_6'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_6']
                df_authors_ind.loc[i, 'topic_7'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_7']
                df_authors_ind.loc[i, 'topic_8'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_8']
                df_authors_ind.loc[i, 'topic_9'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_9']

    for i in range (df_authors_ind.shape[0]):
        summ = df_authors_ind.loc[i,'topic_1'] + df_authors_ind.loc[i,'topic_2'] + df_authors_ind.loc[i,'topic_3'] + df_authors_ind.loc[i,'topic_4'] + df_authors_ind.loc[i,'topic_5'] + df_authors_ind.loc[i,'topic_6'] + df_authors_ind.loc[i,'topic_7'] + df_authors_ind.loc[i,'topic_8'] + df_authors_ind.loc[i,'topic_9']
        if summ > 0:
            df_authors_ind.loc[i,'topic_1'] = df_authors_ind.loc[i,'topic_1']/summ
            df_authors_ind.loc[i,'topic_2'] = df_authors_ind.loc[i,'topic_2']/summ
            df_authors_ind.loc[i,'topic_3'] = df_authors_ind.loc[i,'topic_3']/summ
            df_authors_ind.loc[i,'topic_4'] = df_authors_ind.loc[i,'topic_4']/summ
            df_authors_ind.loc[i,'topic_5'] = df_authors_ind.loc[i,'topic_5']/summ
            df_authors_ind.loc[i,'topic_6'] = df_authors_ind.loc[i,'topic_6']/summ
            df_authors_ind.loc[i,'topic_7'] = df_authors_ind.loc[i,'topic_7']/summ
            df_authors_ind.loc[i,'topic_8'] = df_authors_ind.loc[i,'topic_8']/summ
            df_authors_ind.loc[i,'topic_9'] = df_authors_ind.loc[i,'topic_9']/summ

    res = []

    for i in range (df_authors_ind.shape[0]):
        l = df_authors_ind['EIDs'][i].split(';')
        l.remove('')
        res.append((df_authors_ind['Author_ID'][i],set(l)))


    collab_matrix = np.zeros((df_authors_ind.shape[0],df_authors_ind.shape[0]))

    for i in range (len(res)):
        for j in range (len(res)):
            collab_matrix[i,j] = len(res[i][1].intersection(res[j][1]))

    collab_list = []
    for i in range (collab_matrix.shape[0]):
        for j in range (collab_matrix.shape[0]):
            if j > i:
                if collab_matrix[i,j] != 0:
                    collab_list.append((df_authors_ind['Author_ID'][i] + df_authors_ind['Author_ID'][j], df_authors_ind['Author_ID'][i], df_authors_ind['Author_ID'][j], collab_matrix[i,j]))

    df_collab = pd.DataFrame(data = collab_list, columns=['Author_1_2', 'Author_1', 'Author_2', 'number_of_collaborations'])


    df_collab = df_collab.set_index(['Author_1_2'])

    df_authors_ind ['partners'] = ''
    for i in range (collab_matrix.shape[0]):
        for j in range (collab_matrix.shape[0]):
            if i != j:
                if collab_matrix[i,j] != 0:
                    df_authors_ind ['partners'][i] = str (df_authors_ind ['partners'][i]) + ';' + str (df_authors_ind ['Author_ID'][j])


    df_data_set_ind = pd.DataFrame(data = authors_dep, columns=['Author_1_2', 'Author_1', 'Author_2'])

    df_data_set_ind = df_data_set_ind.set_index(['Author_1_2'])

    df_data_set_ind.insert(2,'TENB',0)
    df_data_set_ind.insert(3,'Cog_Dist', '')
    df_data_set_ind.insert(4,'Geo_Dist',0)
    df_data_set_ind.insert(5,'Prov_Border',0)
    df_data_set_ind.insert(6,'NotContig',0)

    # TENB

    res_p = []

    for i in range (df_authors_ind.shape[0]):
        l = df_authors_ind['partners'][i].split(';')
        l.remove('')
        res_p.append((df_authors_ind['Author_ID'][i],set(l)))

    common_partners_matrix = np.zeros((df_authors_ind.shape[0],df_authors_ind.shape[0]))

    for i in range (len(res_p)):
        for j in range (len(res_p)):
            common_partners_matrix[i,j] = len(res_p[i][1].intersection(res_p[j][1]))


    df_common_partners = pd.DataFrame([])
    df_common_partners.insert(0,'Author_1_2','')
    df_common_partners.insert(1,'Author_1','')
    df_common_partners.insert(2,'Author_2','')
    df_common_partners.insert(3,'Common_partners',{})
    df_common_partners.insert(4,'TENB',float)


    for i in range (common_partners_matrix.shape[0]):
        for j in range (common_partners_matrix.shape[0]):
            if j > i:
                if common_partners_matrix[i,j] != 0:
                    df_common_partners = df_common_partners.append({'Author_1_2': df_authors_ind['Author_ID'][i] + df_authors_ind['Author_ID'][j],'Author_1': df_authors_ind['Author_ID'][i], 'Author_2': df_authors_ind['Author_ID'][j], 'Common_partners': res_p[i][1].intersection(res_p[j][1])}, ignore_index = True)


    number_of_articles = []
    for i in range (collab_matrix.shape[0]):
        number_of_articles.append((df_authors_ind['Author_ID'][i], collab_matrix[i,i]))

    df_number_of_articles = pd.DataFrame(data = number_of_articles, columns=['Author', 'number_of_articles'])

    df_number_of_articles_ = df_number_of_articles.set_index(['Author'])

    df_collab_ = df_collab.set_index(['Author_1' , 'Author_2'])


    for i in range (df_common_partners.shape[0]):
        n = len(df_common_partners['Common_partners'][i])
        list_ENB = []
        for j in range (n):
            common_partner = list (df_common_partners['Common_partners'][i])[j]
            d = df_number_of_articles_.loc[common_partner,'number_of_articles']
            num_article_common_partner = int(d)
            Auth_1 = df_common_partners['Author_1'][i]
            try:
                x = df_collab_.loc[(Auth_1,common_partner),'number_of_collaborations']
            except KeyError:
                x = 0
            if x != 0:
                num_collab_Auth_1 = x
            else:
                num_collab_Auth_1 = df_collab_.loc[(common_partner,Auth_1),'number_of_collaborations']
            Auth_2 = df_common_partners['Author_2'][i]
            try:
                y = df_collab_.loc[(Auth_2,common_partner),'number_of_collaborations']
            except KeyError:
                y = 0
            if y != 0:
                num_collab_Auth_2 = y
            else:
                num_collab_Auth_2 = df_collab_.loc[(common_partner,Auth_2),'number_of_collaborations']
            ENB = ((int(num_collab_Auth_1)) * (int(num_collab_Auth_2))) / num_article_common_partner
            list_ENB.append(ENB)
            TENB = sum(list_ENB)
        df_common_partners['TENB'][i] = TENB

    df_common_partners = df_common_partners.set_index(['Author_1_2'])

    for i in df_common_partners.index:
        df_data_set_ind.loc[i,'TENB'] = df_common_partners.loc[i,'TENB']

    df_authors_ind = df_authors_ind.set_index('Author_ID')


    #Cog_Dist

    for i in df_data_set_ind.index:
        a1 = df_authors_ind.loc[df_data_set_ind.loc[i, 'Author_1'], ['topic_1', 'topic_2', 'topic_3', 'topic_4','topic_5', 'topic_6','topic_7', 'topic_8','topic_9']]
        a1=a1.tolist()
        a2 = df_authors_ind.loc[df_data_set_ind.loc[i, 'Author_2'], ['topic_1', 'topic_2', 'topic_3', 'topic_4','topic_5', 'topic_6','topic_7', 'topic_8','topic_9']]
        a2=a2.tolist()
        if a1 != [0,0,0,0,0,0,0,0,0] and a2 != [0,0,0,0,0,0,0,0,0]:
            cor = numpy.corrcoef(a1, a2)
            df_data_set_ind.loc[i, 'Cog_Dist'] = 1 - cor[0][1]

    # Geo_Dist

    import math

    def Geo_Distance (lat_1,lon_1,lat_2,lon_2):
        R = 6373.0
        lat1 = math.radians(lat_1)
        lon1 = math.radians(lon_1)
        lat2 = math.radians(lat_2)
        lon2 = math.radians(lon_2)
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = R * c
        return distance

    dist = []

    for i in df_data_set_ind.index:
        author_1 = df_data_set_ind.Author_1[i]
        author_2 = df_data_set_ind.Author_2[i]
        auth_1_lat = df_authors_ind.Latitude[author_1]
        auth_1_lng = df_authors_ind.Longitude[author_1]
        auth_2_lat = df_authors_ind.Latitude[author_2]
        auth_2_lng = df_authors_ind.Longitude[author_2]

        dist.append((author_1, author_2, Geo_Distance(auth_1_lat,auth_1_lng,auth_2_lat,auth_2_lng)))

    df_geo_dist = pd.DataFrame(data = dist, columns=['Author_1', 'Author_2', 'GeoDist'])

    df_geo_dist.insert(3,'Author_1_2','')

    df_geo_dist.Author_1_2 = df_geo_dist.Author_1 + df_geo_dist.Author_2

    df_geo_dist.set_index('Author_1_2', inplace = True)

    df_data_set_ind.Geo_Dist = df_geo_dist.GeoDist

    df_temp = pd.merge(df_data_set_ind, df_authors_ind, left_on = 'Author_1', right_index = True, how = 'left')
    df_temp.rename({'Province_code':'Province_code_1'}, axis = 1, inplace = True)
    df_temp.drop(['EIDs','partners', 'Aff_ID', 'Latitude', 'Longitude'], axis = 1, inplace = True)
    df_temp2 = pd.merge(df_temp, df_authors_ind, left_on = 'Author_2', right_index = True, how = 'left')
    df_temp2.drop(['EIDs','partners', 'Aff_ID', 'Latitude', 'Longitude'], axis = 1, inplace = True)
    df_temp2.rename({'Province_code':'Province_code_2'}, axis = 1, inplace = True)

    def comparison_(x, y):
        if x == y:
            return 0
        else:
            return 1

    df_temp2.Prov_Border = df_temp2.apply(lambda x: comparison_(x.Province_code_1, x.Province_code_2), axis = 1)

    df_data_set_ind.Prov_Border = df_temp2.Prov_Border

    list_prov = list(df_authors_ind.Province.unique())

    dic_contig = {'Nova Scotia':['New Brunswick'], 
                  'New Brunswick':['Nova Scotia','Quebec'], 
                  'Newfoundland and Labrador':['Quebec'], 
                  'Quebec':['New Brunswick', 'Newfoundland and Labrador', 'Ontario'], 
                  'Ontario':['Quebec', 'Manitoba'], 
                  'Manitoba':['Ontario','Saskatchewan'], 
                  'Saskatchewan':['Manitoba','Alberta'], 
                  'Alberta':['Saskatchewan','British Columbia'], 
                  'British Columbia':['Alberta']}

    df_temp = pd.merge(df_data_set_ind, df_authors_ind, left_on = 'Author_1', right_index = True, how = 'left')
    df_temp.rename({'Province':'Province_1'}, axis = 1, inplace = True)
    df_temp.drop(['EIDs','partners', 'Aff_ID', 'Latitude', 'Longitude'], axis = 1, inplace = True)
    df_temp2 = pd.merge(df_temp, df_authors_ind, left_on = 'Author_2', right_index = True, how = 'left')
    df_temp2.drop(['EIDs','partners', 'Aff_ID', 'Latitude', 'Longitude'], axis = 1, inplace = True)
    df_temp2.rename({'Province':'Province_2'}, axis = 1, inplace = True)

    def contiguity_ (x, y):
        c = dic_contig[x]
        if c.count(y) == 1:
            return 0
        else:
            return 1

    df_temp2.NotContig = df_temp2.apply(lambda x: contiguity_(x.Province_1, x.Province_2), axis = 1)

    df_data_set_ind.NotContig = df_temp2.NotContig

        # Province dummies

    one_hot = pd.get_dummies(df_temp2[['Province_1','Province_2']])
    df_data_set_ind = df_data_set_ind.join(one_hot)

    df_data_set_ = df_data_set_dep.merge(df_data_set_ind, right_index = True, left_index = True, how = 'left')
    df_data_set = pd.concat([df_data_set,df_data_set_])

    
df_data_set.reset_index(inplace = True)
df_data_set.drop(['Author_1_y', 'Author_2_y'], axis = 1, inplace = True)
df_data_set.rename({'Author_1_x' : 'Author_1'}, axis = 1, inplace = True)
df_data_set.rename({'Author_2_x' : 'Author_2'}, axis = 1, inplace = True)
df_data_set['Log_Geo_Dist'] = df_data_set.Geo_Dist.apply(lambda x :math.log1p(x))
df_data_set.fillna(0, inplace = True)

df_data_set.insert(27,'Top_regions',0)

def Top_regions (reg1, reg2):
    if reg1 == 1 and reg2 == 1:
        return 1
        
df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x.Province_1_Quebec, x.Province_2_Quebec), axis = 1)
df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x.Province_1_Ontario, x.Province_2_Ontario), axis = 1)
df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x['Province_1_British Columbia'], x['Province_2_British Columbia']), axis = 1)

df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x.Province_1_Quebec, x.Province_2_Ontario), axis = 1)
df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x.Province_1_Ontario, x.Province_2_Quebec), axis = 1)

df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x.Province_1_Quebec, x['Province_2_British Columbia']), axis = 1)
df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x['Province_1_British Columbia'], x.Province_2_Quebec), axis = 1)

df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x.Province_1_Ontario, x['Province_2_British Columbia']), axis = 1)
df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x['Province_1_British Columbia'], x.Province_2_Ontario), axis = 1)

df_data_set.Top_regions.fillna(0, inplace = True)

df_data_set.Cog_Dist = df_data_set.Cog_Dist.map(lambda x: None if x == '' else x)


df_data_set.insert(28,'dummy_Quebec',0)
df_data_set.insert(29,'dummy_Ontario',0)
df_data_set.insert(30,'dummy_British_Columbia',0)
df_data_set.insert(31,'dummy_Alberta',0)
df_data_set.insert(32,'dummy_Manitoba',0)
df_data_set.insert(33,'dummy_Nova_Scotia',0)
df_data_set.insert(34,'dummy_Saskatchewan',0)
df_data_set.insert(35,'dummy_New_Brunswick',0)



def dummy_province_ (reg1, reg2):
    if reg1 == 1 or reg2 == 1:
        return 1

    
df_data_set.dummy_Quebec = df_data_set.apply(lambda x: dummy_province_(x.Province_1_Quebec, x.Province_2_Quebec), axis = 1)
df_data_set.dummy_Ontario = df_data_set.apply(lambda x: dummy_province_(x.Province_1_Ontario, x.Province_2_Ontario), axis = 1)
df_data_set.dummy_British_Columbia = df_data_set.apply(lambda x: dummy_province_(x['Province_1_British Columbia'], x['Province_2_British Columbia']), axis = 1)
df_data_set.dummy_Alberta = df_data_set.apply(lambda x: dummy_province_(x.Province_1_Alberta, x.Province_2_Alberta), axis = 1)
df_data_set.dummy_Manitoba = df_data_set.apply(lambda x: dummy_province_(x.Province_1_Manitoba, x.Province_2_Manitoba), axis = 1)
df_data_set.dummy_Nova_Scotia = df_data_set.apply(lambda x: dummy_province_(x['Province_1_Nova Scotia'], x['Province_2_Nova Scotia']), axis = 1)
df_data_set.dummy_Saskatchewan = df_data_set.apply(lambda x: dummy_province_(x.Province_1_Saskatchewan, x.Province_2_Saskatchewan), axis = 1)
df_data_set.dummy_New_Brunswick = df_data_set.apply(lambda x: dummy_province_(x['Province_1_New Brunswick'], x['Province_2_New Brunswick']), axis = 1)

df_data_set.dummy_Quebec.fillna(0, inplace = True)
df_data_set.dummy_Ontario.fillna(0, inplace = True)
df_data_set.dummy_British_Columbia.fillna(0, inplace = True)
df_data_set.dummy_Alberta.fillna(0, inplace = True)
df_data_set.dummy_Manitoba.fillna(0, inplace = True)
df_data_set.dummy_Nova_Scotia.fillna(0, inplace = True)
df_data_set.dummy_Saskatchewan.fillna(0, inplace = True)
df_data_set.dummy_New_Brunswick.fillna(0, inplace = True)

df_data_set['Log_Geo_Dist X TENB'] = df_data_set['Log_Geo_Dist'] * df_data_set['TENB']
df_data_set['Log_Geo_Dist_Sq'] = df_data_set['Log_Geo_Dist'] * df_data_set['Log_Geo_Dist']
df_data_set['Log_Geo_Dist_Sq X TENB'] = df_data_set['Log_Geo_Dist_Sq'] * df_data_set['TENB']


Enlarging the screen is done!
Libraries were imported successfully!
Loading data from sql is done!


Preparing the data set:   0%|          | 0/16 [00:00<?, ?it/s]

In [2]:
X = df_data_set[['TENB', 'Cog_Dist', 'Log_Geo_Dist', 'Prov_Border', 'Top_regions','NotContig', 'Log_Geo_Dist X TENB', 'Log_Geo_Dist_Sq', 'Log_Geo_Dist_Sq X TENB',
                'dummy_Quebec', 'dummy_Ontario',
                'dummy_British_Columbia', 'dummy_Alberta', 'dummy_Manitoba',
                'dummy_Nova_Scotia', 'dummy_Saskatchewan', 'dummy_New_Brunswick']]

X.corr()


,TENB,Cog_Dist,Log_Geo_Dist,Prov_Border,Top_regions,NotContig,Log_Geo_Dist X TENB,Log_Geo_Dist_Sq,Log_Geo_Dist_Sq X TENB,dummy_Quebec,dummy_Ontario,dummy_British_Columbia,dummy_Alberta,dummy_Manitoba,dummy_Nova_Scotia,dummy_Saskatchewan,dummy_New_Brunswick
TENB,1.000000,-0.421814,-0.467064,-0.316806,-0.027308,0.127710,0.437271,-0.377707,0.308548,-0.196166,-0.170287,-0.065704,0.153842,0.036910,-0.058311,-0.063163,0.043542
Cog_Dist,-0.421814,1.000000,0.326030,0.199841,0.034830,-0.088174,-0.184230,0.273359,-0.130569,0.064216,0.144296,0.019174,0.024299,-0.130228,-0.047158,0.027696,-0.006658
Log_Geo_Dist,-0.467064,0.326030,1.000000,0.779708,0.106579,-0.044068,-0.028460,0.971230,0.033354,0.007137,0.192525,0.199132,0.239586,-0.001288,0.098664,0.152611,0.052280
Prov_Border,-0.316806,0.199841,0.779708,1.000000,0.077619,-0.362994,-0.037491,0.783819,0.014034,0.069314,0.267841,0.186753,0.227272,0.052587,0.156819,0.162866,0.115931
Top_regions,-0.027308,0.034830,0.106579,0.077619,1.000000,0.076431,-0.011897,0.134066,-0.008414,-0.165440,0.146841,0.415625,-0.096467,-0.025821,-0.034897,-0.037801,-0.024410
NotContig,0.127710,-0.088174,-0.044068,-0.362994,0.076431,1.000000,0.055638,0.083269,0.039351,-0.249748,-0.433295,-0.019122,0.275536,0.050202,0.145234,0.076076,-0.108803
Log_Geo_Dist X TENB,0.437271,-0.184230,-0.028460,-0.037491,-0.011897,0.055638,1.000000,-0.014789,0.970669,-0.069146,-0.076500,-0.028625,0.066845,-0.018257,-0.025404,-0.027518,0.261047
Log_Geo_Dist_Sq,-0.377707,0.273359,0.971230,0.783819,0.134066,0.083269,-0.014789,1.000000,0.043562,-0.042351,0.119775,0.230939,0.345534,0.012084,0.087407,0.174515,0.031797
Log_Geo_Dist_Sq X TENB,0.308548,-0.130569,0.033354,0.014034,-0.008414,0.039351,0.970669,0.043562,1.000000,-0.064005,-0.056773,-0.020245,0.065374,-0.013292,-0.017967,-0.019462,0.282274
dummy_Quebec,-0.196166,0.064216,0.007137,0.069314,-0.165440,-0.249748,-0.069146,-0.042351,-0.064005,1.000000,-0.283584,-0.168892,-0.354404,-0.197158,-0.091604,-0.077084,-0.066690


In [3]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectFromModel
ros = RandomOverSampler()
scl = StandardScaler()
smt = SMOTE()
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [8]:
# Scenario 1 : No preprocessing


X1 = df_data_set[['Log_Geo_Dist']]
X2 = df_data_set[['Log_Geo_Dist','Cog_Dist']]
X3 = df_data_set[['Log_Geo_Dist','Cog_Dist','Prov_Border','Top_regions','NotContig']]

X4 = df_data_set[['Log_Geo_Dist','Cog_Dist','Prov_Border','Top_regions','NotContig',
                'dummy_Quebec', 'dummy_Ontario', 'dummy_British_Columbia', 'dummy_Alberta', 'dummy_Manitoba', 'dummy_Nova_Scotia', 'dummy_Saskatchewan']]

X5 = df_data_set[['TENB','Log_Geo_Dist','Cog_Dist','Prov_Border','Top_regions','NotContig',
                'dummy_Quebec', 'dummy_Ontario','dummy_British_Columbia', 'dummy_Alberta', 'dummy_Manitoba','dummy_Nova_Scotia', 'dummy_Saskatchewan']]

X6 = df_data_set[['TENB','Log_Geo_Dist','Cog_Dist','Prov_Border','Top_regions','NotContig']]
X7 = df_data_set[['TENB','Log_Geo_Dist','Cog_Dist', 'Log_Geo_Dist X TENB']]
X8 = df_data_set[['TENB','Log_Geo_Dist','Cog_Dist', 'Log_Geo_Dist X TENB','Log_Geo_Dist_Sq X TENB']]

X9 = df_data_set[['Log_Geo_Dist','Cog_Dist','Top_regions','NotContig']]
X10 = df_data_set[['Log_Geo_Dist','Cog_Dist','Top_regions','NotContig',
                'dummy_Quebec', 'dummy_Ontario','dummy_British_Columbia', 'dummy_Alberta', 'dummy_Manitoba','dummy_Nova_Scotia', 'dummy_Saskatchewan']]

X11 = df_data_set[['TENB','Log_Geo_Dist','Cog_Dist']]




X1 = sm.tools.tools.add_constant(X1, prepend=True, has_constant='add')
X2 = sm.tools.tools.add_constant(X2, prepend=True, has_constant='add')
X3 = sm.tools.tools.add_constant(X3, prepend=True, has_constant='add')
X4 = sm.tools.tools.add_constant(X4, prepend=True, has_constant='add')
X5 = sm.tools.tools.add_constant(X5, prepend=True, has_constant='add')
X6 = sm.tools.tools.add_constant(X6, prepend=True, has_constant='add')
X7 = sm.tools.tools.add_constant(X7, prepend=True, has_constant='add')
X8 = sm.tools.tools.add_constant(X8, prepend=True, has_constant='add')
X9 = sm.tools.tools.add_constant(X9, prepend=True, has_constant='add')
X10 = sm.tools.tools.add_constant(X10, prepend=True, has_constant='add')
X11 = sm.tools.tools.add_constant(X11, prepend=True, has_constant='add')


X = [X7]

y = df_data_set['collaboration_binary']

for X in X:
    logit_model=sm.Logit(y, X)
    result=logit_model.fit(method_kwargs={"warn_convergence": False})
    print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.052667
         Iterations 12
                           Results: Logit
Model:              Logit                Pseudo R-squared: 0.772     
Dependent Variable: collaboration_binary AIC:              114.8081  
Date:               2021-08-16 23:47     BIC:              139.3218  
No. Observations:   995                  Log-Likelihood:   -52.404   
Df Model:           4                    LL-Null:          -229.39   
Df Residuals:       990                  LLR p-value:      2.4228e-75
Converged:          1.0000               Scale:            1.0000    
No. Iterations:     12.0000                                          
---------------------------------------------------------------------
                      Coef.  Std.Err.    z    P>|z|   [0.025   0.975]
---------------------------------------------------------------------
const                 1.0101   0.4425  2.2826 0.0225   0.1428  1.8775
TENB           